In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load Census Bureau Household Pulse Survey data
hps = pd.read_csv("hps_data.csv")
hps.head()

,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,RHISPANIC,...,PSWHYCHG1,PSWHYCHG2,PSWHYCHG3,PSWHYCHG4,PSWHYCHG5,PSWHYCHG6,PSWHYCHG7,PSWHYCHG8,PSWHYCHG9,INCOME
0,V340000001S34010804300113,34,1,NaN,2,1548.941305,2889.966484,1986,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,7
1,V340000001S37010632600113,34,1,NaN,2,1080.178856,3023.046143,1967,2,2,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,4
2,V340000001S52011057710113,34,1,NaN,2,1542.979030,4318.263387,1941,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,4
3,V340000001S79010365210123,34,1,NaN,2,1111.825305,2074.409055,1962,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,8
4,V340000002S01021059400123,34,2,NaN,4,87.544682,173.361259,1975,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88,-88,5


In [3]:
# Load CDC data
cdc = pd.read_csv("COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv")
cdc.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Additional_Doses_18Plus,Additional_Doses_18Plus_Vax_Pct,Additional_Doses_50Plus,Additional_Doses_50Plus_Vax_Pct,Additional_Doses_65Plus,Additional_Doses_65Plus_Vax_Pct,Additional_Doses_Moderna,Additional_Doses_Pfizer,Additional_Doses_Janssen,Additional_Doses_Unk_Manuf
0,11/16/2021,46,HI,2738800,111700,1025160,1601940,0,193436,226140,...,102827.0,12.8,80786.0,18.4,58275.0,24.6,24145.0,78252.0,570.0,2.0
1,11/16/2021,46,IL,20805235,1033300,7663400,12108535,0,164185,191969,...,1384001.0,19.2,1066406.0,28.7,745263.0,41.9,528502.0,844422.0,12021.0,457.0
2,11/16/2021,46,OH,17728785,833000,6991380,9904405,0,151669,177275,...,1191580.0,20.6,977371.0,29.1,722517.0,41.5,465407.0,710412.0,16840.0,222.0
3,11/16/2021,46,TX,47765235,2325500,17998260,27441475,0,164731,198187,...,2236905.0,15.5,1695704.0,24.9,1101393.0,35.8,951389.0,1264475.0,25356.0,68.0
4,11/16/2021,46,BP2,285240,15600,122220,147420,0,0,0,...,8686.0,7.0,3062.0,10.9,548.0,15.4,4184.0,4386.0,96.0,20.0


The Census Bureau website indicates that our data was generated during week 34 of phase 3.2. This means that this specific data was collected on July 21st, 2021. So, we're going to subset our CDC data to 7/21/2021 before we merge it with the HPS data.

In [4]:
# Subset CDC data
cdc_new = cdc[cdc["Date"] == '7/21/2021']
cdc_new.head()

,Date,MMWR_week,Location,Distributed,Distributed_Janssen,Distributed_Moderna,Distributed_Pfizer,Distributed_Unk_Manuf,Dist_Per_100K,Distributed_Per_100k_12Plus,...,Additional_Doses_18Plus,Additional_Doses_18Plus_Vax_Pct,Additional_Doses_50Plus,Additional_Doses_50Plus_Vax_Pct,Additional_Doses_65Plus,Additional_Doses_65Plus_Vax_Pct,Additional_Doses_Moderna,Additional_Doses_Pfizer,Additional_Doses_Janssen,Additional_Doses_Unk_Manuf
7644,7/21/2021,29,CT,4828735,250000,1957680,2621055,0,135437,155467,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7645,7/21/2021,29,NM,2413065,138500,1050040,1224525,0,115082,135002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7646,7/21/2021,29,IN,6852890,427500,2734160,3691230,0,101792,120120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7647,7/21/2021,29,ME,1887110,103100,811560,972450,0,140388,159258,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7648,7/21/2021,29,TX,32549095,1894600,13134540,17519955,0,112254,135052,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


When we merge the two datasets, we are going to merge on the variable **state**. Now, while both of the datasets contain state-level information, they do so under different variable names and formats. The Census Bureau variable is called **EST_ST** and is recorded in the dataframe using numbers that correspond to states that are recorded in the codebook. The CDC variable is called **Location** and records state abbreviations. Our next step is to assign copies of these variables named state in each dataframe, and then create a dictionary containing the information from the HPS codebook and replace the numeric values in the HPS dataframe with state abbreviations to match the information in the CDC dataframe.

In [5]:
# Unique values of states from the HPS dataframe
hps.EST_ST.unique()

array([ 1,  2,  6,  4,  5,  8, 37,  9, 10, 12, 13, 15, 24, 16, 26, 46, 29,
       17, 18, 53, 19, 48, 42, 25, 34, 45, 36, 33, 55, 20, 40, 21, 31, 41,
       22, 23, 47, 27, 51, 28, 30, 49, 32, 39, 35, 38, 44, 56, 11, 50, 54],
      dtype=int64)

In [6]:
# Unique values of states from the CDC dataframe
cdc_new.Location.unique()

array(['HI', 'IL', 'OH', 'TX', 'BP2', 'US', 'MN', 'FL', 'UT', 'PR', 'ME',
       'VA2', 'CA', 'NC', 'AR', 'RP', 'MH', 'CT', 'MP', 'DC', 'KY', 'FM',
       'MS', 'DE', 'VA', 'GA', 'CO', 'NJ', 'AS', 'WI', 'PA', 'IN', 'TN',
       'NE', 'SD', 'NH', 'OR', 'NM', 'DD2', 'ID', 'AK', 'GU', 'VT', 'MD',
       'AL', 'OK', 'WY', 'KS', 'NY', 'IA', 'IH2', 'SC', 'MT', 'MI', 'WA',
       'AZ', 'WV', 'VI', 'MA', 'ND', 'NV', 'RI', 'MO', 'LA', 'LTC'],
      dtype=object)

In [7]:
hps = hps.assign(state = hps.EST_ST)

In [15]:
cdc_new = cdc_new.assign(state = cdc_new.Location)

Index(['Date', 'MMWR_week', 'Location', 'Distributed', 'Distributed_Janssen',
       'Distributed_Moderna', 'Distributed_Pfizer', 'Distributed_Unk_Manuf',
       'Dist_Per_100K', 'Distributed_Per_100k_12Plus',
       'Distributed_Per_100k_18Plus', 'Distributed_Per_100k_65Plus',
       'Administered', 'Administered_12Plus', 'Administered_18Plus',
       'Administered_65Plus', 'Administered_Janssen', 'Administered_Moderna',
       'Administered_Pfizer', 'Administered_Unk_Manuf', 'Admin_Per_100K',
       'Admin_Per_100k_12Plus', 'Admin_Per_100k_18Plus',
       'Admin_Per_100k_65Plus', 'Recip_Administered',
       'Administered_Dose1_Recip', 'Administered_Dose1_Pop_Pct',
       'Administered_Dose1_Recip_12Plus',
       'Administered_Dose1_Recip_12PlusPop_Pct',
       'Administered_Dose1_Recip_18Plus',
       'Administered_Dose1_Recip_18PlusPop_Pct',
       'Administered_Dose1_Recip_65Plus',
       'Administered_Dose1_Recip_65PlusPop_Pct', 'Series_Complete_Yes',
       'Series_Complete_Pop_

In [9]:
# Replace values in hps using a dictionary created from value of states in hps codebook
states_key = {1:'AL', 2:'AK', 4:'AZ', 5:'AR', 6:'CA', 8:'CO', 9:'CT', 10:'DE', 11:'DC', 12:'FL', 13:'GA', 15:'HI',16:'ID', 
              17:'IL', 18:'IN', 19:'IA', 20:'KS', 21:'KY', 22:'LA', 23:'ME', 24:'MD', 25:'MA', 26:'MI', 27:'MN', 28:'MS',
              29:'MO', 30:'MT', 31:'NE', 32:'NV', 33:'NH', 34:'NJ', 35:'NM', 36:'NY', 37:'NC', 38:'ND', 39:'OH', 40:'OK',
              41:'OR', 42:'PA', 44:'RI', 45:'SC', 46:'SD', 47:'TN', 48:'TX', 49:'UT', 50:'VT', 51:'VA', 53:'WA', 54:'WV',
              55:'WI', 56:'WY'}

hps.state.replace(states_key, inplace=True)

In [19]:
hps.state

0        AL
1        AL
2        AL
3        AL
4        AK
         ..
64557    WI
64558    WY
64559    WY
64560    FL
64561    ID
Name: state, Length: 64562, dtype: object

In [18]:
# Choose variables needed from cdc_new
cdc_vars = cdc_new[['state','Distributed','Dist_Per_100K','Distributed_Per_100k_12Plus', 'Distributed_Per_100k_18Plus', 
                   'Distributed_Per_100k_65Plus', 'Administered', 'Administered_12Plus', 'Administered_18Plus', 
                   'Administered_65Plus','Admin_Per_100K', 'Admin_Per_100k_12Plus', 'Admin_Per_100k_18Plus',
                   'Admin_Per_100k_65Plus']]

# Merge dataframes together
combined_df = hps.merge(cdc_vars, how="left", on="state")

combined_df.head()

,SCRAM,WEEK,EST_ST,EST_MSA,REGION,HWEIGHT,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,RHISPANIC,...,Distributed_Per_100k_18Plus,Distributed_Per_100k_65Plus,Administered,Administered_12Plus,Administered_18Plus,Administered_65Plus,Admin_Per_100K,Admin_Per_100k_12Plus,Admin_Per_100k_18Plus,Admin_Per_100k_65Plus
0,V340000001S34010804300113,34,1,NaN,2,1548.941305,2889.966484,1986,2,1,...,129374,580755,3485659,3485337,3390541,1211332,71090,83244,88877,142537
1,V340000001S37010632600113,34,1,NaN,2,1080.178856,3023.046143,1967,2,2,...,129374,580755,3485659,3485337,3390541,1211332,71090,83244,88877,142537
2,V340000001S52011057710113,34,1,NaN,2,1542.979030,4318.263387,1941,2,1,...,129374,580755,3485659,3485337,3390541,1211332,71090,83244,88877,142537
3,V340000001S79010365210123,34,1,NaN,2,1111.825305,2074.409055,1962,2,1,...,129374,580755,3485659,3485337,3390541,1211332,71090,83244,88877,142537
4,V340000002S01021059400123,34,2,NaN,4,87.544682,173.361259,1975,2,1,...,153469,924220,688769,686698,647234,142364,94153,112781,117346,155440
